<a href="https://colab.research.google.com/github/GyanLevy/Cloud_Computing/blob/main/Students.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [6]:
# === Students Form (Colab) — uses `with open(...)` to read the file ===
import re, ast, os, json
from IPython.display import display, HTML, clear_output

# --- היכן נמצא הקובץ? ---
# אם העלית ל-Colab (בחלון Files), הנתיב הוא בד"כ /content/students.txt
PREFERRED_PATHS = ["/content/students.txt", "students.txt", "/content/1.txt", "1.txt"]

file_path = None
for p in PREFERRED_PATHS:
    if os.path.exists(p):
        file_path = p
        break
if file_path is None:
    raise FileNotFoundError(
        "לא נמצא students.txt. העלה את הקובץ ל-Colab (בחלון Files) או עדכן את הנתיב ברשימת PREFERRED_PATHS."
    )

# --- קריאה באמצעות with open(...) בדיוק כמו שביקשת ---
with open(file_path, "r", encoding="utf-8") as f:
    raw_text = f.read()

# אפשר גם לעבוד לפי שורות:
lines = [ln.strip() for ln in raw_text.splitlines() if ln.strip()]

# --- פרסינג של כל שורה: First, Last, Email, ["Course1","Course2",...], Link ---
pattern = re.compile(r'^\s*([^,]+),\s*([^,]+),\s*([^,]+),\s*(\[.*\]),\s*(\S+)\s*$')
records = []
for idx, line in enumerate(lines, 1):
    m = pattern.match(line)
    if not m:
        raise ValueError(f"שורה {idx} לא בפורמט המצופה:\n{line}")
    first, last, email, courses_str, link = m.groups()
    try:
        courses = ast.literal_eval(courses_str)
    except Exception:
        # fallback פשוט
        courses = [c.strip().strip('"').strip("'") for c in courses_str.strip("[]").split(",") if c.strip()]
    records.append({"first_name": first, "last_name": last, "email": email, "courses": courses, "link": link})

# --- CSS לעיצוב הכרטיס והטופס ---
CARD_CSS = """
<style>
:root {
  --bg: #0b0f14; --card: #121923; --card-2: #0f141b; --text: #e7eef7; --muted: #b6c6da;
  --accent: #3ea6ff; --chip: #1e2a38; --chip-text: #d9e6f7; --shadow: 0 10px 30px rgba(0,0,0,.35);
}
@media (prefers-color-scheme: light) {
  :root {
    --bg: #f6f8fb; --card: #ffffff; --card-2: #f3f6fa; --text: #0e141b; --muted: #3b506a;
    --accent: #0b67d3; --chip: #e9f2ff; --chip-text: #0b3d7a; --shadow: 0 10px 30px rgba(0,0,0,.08);
  }
}
.wrapper { max-width: 900px; font-family: ui-sans-serif, system-ui, -apple-system, "Segoe UI", Roboto, Arial; color: var(--text); }
.header { display:flex; align-items:center; gap:12px; margin: 8px 0 14px; }
.header h1 { font-size: 20px; margin: 0; letter-spacing:.2px; }
.card {
  margin-top: 14px; padding: 18px 20px; border-radius: 14px;
  background: linear-gradient(180deg, var(--card) 0%, var(--card-2) 100%);
  border: 1px solid rgba(255,255,255,.06); color: var(--text); box-shadow: var(--shadow);
}
.card h2 { margin: 0 0 8px 0; font-size: 22px; }
.kv { margin: 6px 0; }
.kv b { display:inline-block; width:100px; }
.chips { display:flex; flex-wrap:wrap; gap:8px; margin:10px 0 16px; }
.chip { background: var(--chip); color: var(--chip-text); padding:6px 10px; border-radius:999px; font-size:12px; border: 1px solid rgba(255,255,255,.08); }
a.btn { text-decoration:none; background: var(--accent); color: white; padding: 10px 14px; border-radius: 12px; font-weight: 700; }
.small { color: var(--muted); font-size: 12px; margin-top: 8px; }
</style>
"""

def student_card_html(s):
    chips = "".join(f'<span class="chip">{c}</span>' for c in s["courses"])
    return f"""
    <div class="card">
      <h2>{s['first_name']} {s['last_name']}</h2>
      <div class="kv"><b>Email</b> <a href="mailto:{s['email']}">{s['email']}</a></div>
      <div class="kv"><b>Courses</b></div>
      <div class="chips">{chips}</div>
      <div><a class="btn" href="{s['link']}" target="_blank" rel="noopener">Open interesting link ↗</a></div>
      <div class="small">רמז: אפשר לבחור סטודנט אחר בתיבה למעלה.</div>
    </div>
    """

# --- ניסיון להשתמש ב־ipywidgets; אם אין — נציג Fallback HTML אינטראקטיבי ---
use_widgets = False
try:
    import ipywidgets as widgets
    use_widgets = True
except Exception:
    use_widgets = False

if use_widgets:
    dd = widgets.Dropdown(
        options=[(f"{r['first_name']} {r['last_name']}  ·  {r['email']}", i) for i, r in enumerate(records)],
        value=0, description="סטודנט:", layout=widgets.Layout(width="650px")
    )
    out = widgets.Output()

    def render(i):
        with out:
            clear_output(wait=True)
            display(HTML(CARD_CSS + '<div class="wrapper">' + student_card_html(records[i]) + '</div>'))

    def on_change(ch):
        if ch["name"] == "value" and ch["type"] == "change":
            render(ch["new"])

    dd.observe(on_change)
    display(dd); render(0)
else:
    # Fallback HTML בלבד (עם כפתורים לכל סטודנט)
    buttons_html = ""
    for i, s in enumerate(records):
        buttons_html += f"""
        <button onclick="selectStudent({i})" style="margin:4px; padding:8px 12px; border-radius:10px; border:1px solid #ccc; cursor:pointer;">
            {s['first_name']} {s['last_name']} • {s['email']}
        </button>
        """
    html = f"""
    {CARD_CSS}
    <div class="wrapper">
      <div class="header"><h1>טופס בחירה - סטודנטים</h1></div>
      <div>{buttons_html}</div>
      <div id="details">{student_card_html(records[0])}</div>
    </div>
    <script>
      const STUDENTS = {json.dumps(records)};
      const chip = (t) => `<span class="chip">${{t}}</span>`;
      function renderCard(i){{
        const s = STUDENTS[i];
        const chips = (s.courses||[]).map(chip).join('');
        return `
          <div class="card">
            <h2>${{s.first_name}} ${{s.last_name}}</h2>
            <div class="kv"><b>Email</b> <a href="mailto:${{s.email}}">${{s.email}}</a></div>
            <div class="kv"><b>Courses</b></div>
            <div class="chips">${{chips}}</div>
            <div><a class="btn" href="${{s.link}}" target="_blank" rel="noopener">Open interesting link ↗</a></div>
            <div class="small">רמז: לחץ על כפתור אחר כדי להחליף.</div>
          </div>`;
      }}
      function selectStudent(i){{
        document.querySelector("#details").innerHTML = renderCard(i);
      }}
    </script>
    """
    display(HTML(html))


ValueError: שורה 1 לא בפורמט המצופה:
One